<a href="https://colab.research.google.com/github/xiongjeffrey/CCLE_Depression_Analysis/blob/main/NeuralEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Reference paper: https://github.com/geomstats/challenge-iclr-2021/blob/main/gcorso__Neural-Sequence-Distance-Embeddings/Neural_SEED.ipynb

In [ ]:
# INITIAL SET UP: ONLY NEED TO RUN ONCE PER RUNTIME

!pip3 install geomstats
!apt install clustalw
!pip install biopython
!pip install python-Levenshtein
!pip install Cython
!pip install networkx
!pip install tqdm
!pip install gdown
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/gcorso/neural_seed.git
import os
os.chdir("neural_seed")
!cd hierarchical_clustering/relaxed/mst; python setup.py build_ext --inplace; cd ../unionfind; python setup.py build_ext --inplace; cd ..; cd ..; cd ..;
os.environ['GEOMSTATS_BACKEND'] = 'pytorch'

!pip install keras

In [ ]:
# INITIAL SET UP CONTINUED: ONLY NEED ONCE PER RUNTIME
import torch
import os 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

import sys
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from Levenshtein import distance as levenshtein_distance

from edit_distance.train import load_edit_distance_dataset
from util.data_handling.data_loader import get_dataloaders
from util.ml_and_math.loss_functions import AverageMeter

In [ ]:
# from geomstats.geometry.poincare_ball import PoincareBall

In [ ]:
# IMPORT DATASET HERE
# commented-out portion is reference from neuralseed

!wget https://github.com/bhattlab/SupplementaryInformation/raw/master/SmORFinder/datasets.tar.gz
!tar -zxvf datasets.tar.gz -C '/content/'

# !gdown --id 1yZTOYrnYdW9qRrwHSO5eRc8rYIPEVtY2 # for edit distance approximation
# !gdown --id 1hQSHR-oeuS9bDVE6ABHS0SoI4xk3zPnB # for closest string retrieval
# !gdown --id 1ukvUI6gUTbcBZEzTVDpskrX8e6EHqVQg # for hierarchical clustering

In [ ]:
# Data Processing
df = pd.read_csv("/content/datasets/dataset_FINAL.tsv", sep='\t')
df.head()

# save subset of dataset

df_subset=df.sample(10000, random_state=42)
df_subset.to_csv('smorf_subset.csv')

# get only protein sequence from training subset
train=df_subset[df_subset.set=='train']
x=train.smorf.tolist()
y=train.y=='positive'

# encode the protein sequence as a one-hot
def onehote(sequence):
  seq_array = np.array(list(sequence))

  #integer encode the sequence
  label_encoder = LabelEncoder()
  integer_encoded_seq = label_encoder.fit_transform(seq_array) 

  #one hot the sequence
  onehot_encoder = OneHotEncoder(sparse=False)

  #reshape because that's what OneHotEncoder likes
  integer_encoded_seq = integer_encoded_seq.reshape(len(integer_encoded_seq), 1)
  onehot_encoded_seq = onehot_encoder.fit_transform(integer_encoded_seq)
  return onehot_encoded_seq

In [4]:
def levenshteinDist(seq1, seq2): #!TODO: figure out how to implement this properly taking in numerics
  return calculate_levenshtein_distance(str_1, str_2)

[2, 4, 5, 6]


In [5]:
# Data Generation
class DistGenerator():

  def __init__(self, seqA, seqB, func, batchSize = 32):
    self.seqA = seqA
    self.seqB = seqB
    self.batchSize = batchSize
    self.dist = func

  def __data_generation(self, seqA, seqB):
      '''
      Generate data containing batch_size samples.
          
      Inputs: two naked sequences (numpy arrays) + parameter for batch size (default 32)
              
      Returns: tuple of aBatches, bBatches, batchDist
          batchSeqs : tuple of size 2, i.e. (aBatches, bBatches), containing sequences contained in each batch
            each batch sequence will contain an array of numpy arrays containing each sequence
          batchDist: dist between the pairs of samples in the corresponding entries of X0 and X1, array of length batch_size
      '''
      # Initialization
      aBatches = []
      bBatches = []

      batchDist = [0]*self.batchSize

      # Generate data
      # if there's a remainder at the end, it will go in one slightly longer batch
      for i in range(len(seqA)/self.batchSize):

          # Store sample pairings
          aBatches.append(seqA[i*self.batchSize[i][0]:])
          bBatches.append(seqB[i*self.batchSize[i][1]:])

          # Store distance between the pair
          batchDist[i] = self.dist(aBatches[-1], bBatches[-1])

      return (aBatches, bBatches, batchDist)

In [ ]:
# sequence transformation
def seqTransform(seqA, seqB): # takes in onehote
      sequence = np.zeros((max(len(seqA), len(seqB)), 4))
        # WLOG, seqA is longer      
      if len(seqA) < len(seqB):
        temp = seqA
        seqA = seqB
        seqB = temp
      
      firstLimit = len(seqB)
      secondLimit = len(seqA)

      i = 0

      for i in range(firstLimit):
        sequence[i] = np.subtract(seqA[i], seqB[i])
      for i in range(secondLimit):
        sequence[i] = seqA[i]

      # defining basic transformation
      # sequence has dimensions n * 4
      seqTensor = sequence.astype(np.float32)
      seqTensor = torch.tensor(sequence, dtype=torch.float32)
      return seqTensor

In [ ]:
class LevenshteinNN(nn.Module): # Linear layers so far

    def __init__(self, sequence): 
      super(ConvNet, self).__init__()
      self.sequence = sequence
      
      self.base = nn.Sequential( 
          
        #!TODO: think more deeply about architecture!
        # can refer to smORF paper for ideas: https://www.sciencedirect.com/science/article/pii/S1931312820306193
        self.conv1 = nn.Conv1d(len(sequence), 35, 3, stride=2) # in_channels, out_channels, kernel_size, stride=1
        self.drop1 = nn.Dropout(p = 0.1)
        self.pool = nn.MaxPool1d(3, 2)
        self.conv2 = nn.Conv1d(35, 10, 5)
        self.drop2 = nn.Dropout(p = 0.1)
        self.fc1 = nn.Linear(10 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

      )

    def forward(self, features):
      # x = self.layer1(features.float())
      # x = self.layer2(x)
      # x = self.layer3(x)
        
      x = self.base(features.float())

      return x # returns 32 * 1 size matrix

In [ ]:
testHotA = onehote(x[1])
testHotB = onehote(x[2])
testSeq = seqTransform(testHotA, testHotB)

features = torch.from_numpy(testHotB.transpose())

features = features.to("cuda")
test = LevenshteinNN(testSeq)
test.to("cuda")
test(features)

tensor([[-0.3459],
        [-0.3326],
        [-0.3926],
        [-0.3695]], device='cuda:0', grad_fn=<AddmmBackward>)

In [1]:
def hyperbolicLoss(x, y): # finds the hyperbolic distance between two vectors of the same length
  #!TODO: implement distance via this: https://en.wikipedia.org/wiki/Poincar%C3%A9_disk_model#Distance
  sum = 1
  for i in range(len(x)):
    sum += ((y[i] - x[i])**2)/(2*x[-1]*y[-1])
  return torch.arccosh(sum)

In [ ]:
class LinearEncoder(nn.Module):
    """  Linear model which simply flattens the sequence and applies a linear transformation. """

    def __init__(self, len_sequence, embedding_size, alphabet_size=4):
        super(LinearEncoder, self).__init__()
        self.encoder = nn.Linear(in_features=alphabet_size * len_sequence, 
                                 out_features=embedding_size)

    def forward(self, sequence):
        # flatten sequence and apply layer
        B = sequence.shape[0]
        sequence = sequence.reshape(B, -1)
        emb = self.encoder(sequence)
        return emb


class PairEmbeddingDistance(nn.Module):
    """ Wrapper model for a general encoder, computes pairwise distances and applies projections """

    def __init__(self, embedding_model, embedding_size, scaling=False):
        super(PairEmbeddingDistance, self).__init__()
        self.hyperbolic_metric = PoincareBall(embedding_size).metric.dist
        self.embedding_model = embedding_model
        self.radius = nn.Parameter(torch.Tensor([1e-2]), requires_grad=True)
        self.scaling = nn.Parameter(torch.Tensor([1.]), requires_grad=True)

    def normalize_embeddings(self, embeddings):
        """ Project embeddings to an hypersphere of a certain radius """
        min_scale = 1e-7
        max_scale = 1 - 1e-3
        return F.normalize(embeddings, p=2, dim=1) * self.radius.clamp_min(min_scale).clamp_max(max_scale)

    def encode(self, sequence):
        """ Use embedding model and normalization to encode some sequences. """
        enc_sequence = self.embedding_model(sequence)
        enc_sequence = self.normalize_embeddings(enc_sequence)
        return enc_sequence

    def forward(self, sequence):
        # flatten couples
        (B, _, N, _) = sequence.shape
        sequence = sequence.reshape(2 * B, N, -1)

        # encode sequences
        enc_sequence = self.encode(sequence)

        # compute distances
        enc_sequence = enc_sequence.reshape(B, 2, -1)
        distance = self.hyperbolic_metric(enc_sequence[:, 0], enc_sequence[:, 1])
        distance = distance * self.scaling

        return distance

In [ ]:
def train(model, data, optimizer, loss, device):
  # model is the particular model we are training on (e.g. Levenshtein)
  # data is the set of data that we're working on (e.g. reads) AND the target (e.g. real Levenshtein distance)
  # optimizer = optim.Adam(model.parameters(), lr=0.001)
  # loss is chosen loss function
  # device is usually cuda


  avg_loss = AverageMeter()
  model.train()

  for sequences, target in data:
    # move to device
    sequences, target = sequences.to(device), target.to(device)

    #forward
    optimizer.zero_grad()
    output = model(sequences)

    # loss and backpropagation
    loss_train = loss(output, target)
    loss_train.backward()
    optimizer.step()

  # keep track of average loss
  avg_loss.update(loss_train.data.item(), sequences.shape[0])

def test(model, data, loss, device):
  avg_loss = AverageMeter()
  model.eval()

  for sequences, target in data:
    # move examples to right device
    sequences, target = sequences.to(device), target.to(device)

    # forward propagation and loss computation
    output = model(sequences)
    loss_val = loss(output, target).data.item()
    avg_loss.update(loss_val, sequences.shape[0])

  return avg_loss.avg


In [ ]:
# Verification of results: via https://github.com/python-engineer/pytorchTutorial/blob/master/14_cnn.py
with torch.no_grad():

    n_correct = 0
    n_samples = 0

    trialNum = 0 # can be set to desired number of verification trials per item

    n_class_correct = [0 for i in range(trialNum)]
    n_class_samples = [0 for i in range(trialNum)]

    for seqs, targets in loader:

        seqs = seqs.to(device)
        labels = labels.to(device)
        outputs = LevenshteinNN(seqs)

        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += targets.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            target = targets[i]
            pred = predicted[i]
            if (target == pred):
                n_class_correct[target] += 1
            n_class_samples[target] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

In [ ]:
# Driver Code
#!TODO: clean up this mess :(
EMBEDDING_SIZE = 128

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(2021)
if device == 'cuda':
    torch.cuda.manual_seed(2021)

# load data
datasets = onehote(x[1])
# datasets = load_edit_distance_dataset('./edit_qiita_large.pkl')
loaders = get_dataloaders(datasets, batch_size=128, workers=1)

# model, optimizer and loss
encoder = LinearEncoder(152, EMBEDDING_SIZE)
model = PairEmbeddingDistance(embedding_model=encoder, embedding_size=EMBEDDING_SIZE)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss = hyperbolicLoss()

# training
for epoch in range(0, 21):
    t = time.time()
    loss_train = train(model, loaders['train'], optimizer, loss, device)
    loss_val = test(model, loaders['val'], loss, device)

    # print progress
    if epoch % 5 == 0:
        print('Epoch: {:02d}'.format(epoch),
              'loss_train: {:.6f}'.format(loss_train),
              'loss_val: {:.6f}'.format(loss_val),
              'time: {:.4f}s'.format(time.time() - t))
      
# testing
for dset in loaders.keys():
    avg_loss = test(model, loaders[dset], loss, device)
    print('Final results {}: loss = {:.6f}'.format(dset, avg_loss))

AttributeError: ignored

In [ ]:
from hierarchical_clustering.unsupervised.unsupervised import hierarchical_clustering_testing

hierarchical_clustering_testing(encoder_model=model, data_path='./hc_qiita_large_extr.pkl',
                                batch_size=128, device=device, distance='hyperbolic')

In [ ]:
from multiple_alignment.guide_tree.guide_tree import approximate_guide_trees

# performs neighbour joining algorithm on the estimate of the pairwise distance matrix
approximate_guide_trees(encoder_model=model, dataset=datasets['test'],
                        batch_size=128, device=device, distance='hyperbolic')

# Command line clustalw using the tree generated with the previous command. 
# The substitution matrix and gap penalties are set to simulate the classical edit distance used to train the model 
!clustalw -infile="sequences.fasta" -dnamatrix=multiple_alignment/guide_tree/matrix.txt -transweight=0 -type='DNA' -gapopen=1 -gapext=1 -gapdist=10000 -usetr